# Анализ сервиса "Яндекс Книги"

## ВВедение

    Необходимо проанализировать данные таблиц  (source_db.audition) и (source_db.content) из сервиса Яндекс Книги и решить задачи от менеджера на их основе. Таблицы этого проекта содержат данные о чтении и прослушивании контента в сервисе Яндекс Книги, которые включают информацию о пользователях, платформах, времени, длительности сессий и типах контента. Данные представлены за период с 1 сентября по 11 декабря 2024 года.  Для решения задач будет использоваться — ClickHouse. 
    
    Таблица bookmate.content содержит данные о контенте и состоит из следующих полей:
    main_content_id — идентификатор основного контента;
    main_author_id — идентификатор основного автора контента;
    main_content_type — тип контента;
    main_content_name— название контента;
    main_content_duration_hours — длительность контента в часах;
    published_topic_title_list — список жанров контента.
    
    Таблица bookmate.audition содержит данные об активности пользователей и состоит из следующих полей:
    audition_id — уникальный идентификатор сессии чтения или прослушивания;
    puid — идентификатор пользователя;
    usage_platform_ru — название платформы, с помощью которой пользователь слушал контент;
    msk_business_dt_str — дата события в формате строки (московское время);
    app_version — версия приложения, которая использовалась для чтения или прослушивания;
    adult_content_flg — был ли это контент для взрослых: True или False;
    hours — длительность чтения или прослушивания в часах;
    hours_sessions_long — продолжительность длинных сессий чтения или прослушивания в часах;
    kids_content_flg — был ли это детский контент: True или False;
    main_content_id — идентификатор основного контента;
    usage_geo_id — идентификатор географического местоположения.
    
 

# Работа с данными бизнеса в ClickHouse

    Выполнил: Калинина Ольга Владимировна
    Дата: 03.10.25

# Данные о таблицах 

### Информация о таблице - source_db.audition;


    name               |type            |
    -------------------+----------------+
    audition_id        |Int32           |
    puid               |String          |
    usage_platform_ru  |String          |
    msk_business_dt_str|Date            |
    app_version        |Nullable(String)|
    adult_content_flg  |Bool            |
    hours              |Float32         |
    hours_sessions_long|Float32         |
    kids_content_flg   |Bool            |
    main_content_id    |String          |
    usage_geo_id_name  |String          |
    usage_country_name |String          |
    ------------------------------------+

    Таблица source_db.audition содержит информацию о пользователях. Есть информация о платформе приложения, времени сессии, времени использования приложения, категории приложения (18+), о городе и стране читателя.  

### Информация о таблице - source_db.content;


    name                       |type         |
    ---------------------------+-------------+
    main_content_id            |String       |
    main_content_type          |String       |
    main_content_name          |String       |
    main_content_duration_hours|Float32      |
    main_author_name           |String       |
    published_topic_title_list |Array(String)|
    -----------------------------------------+
    
    Таблица содержит информацию о контенте. Его название, тип контента, название контента, длительность контента, автор контента, теги с типами жанров.

## Задание 1

    SELECT
            usage_geo_id_name AS name_city, --Название города или области,
            round(sum(hours), 2) AS total_hours, --Общая длительность прочитанного и прослушанного контента, 
            round(sumIf(hours, usage_platform_ru = 'Букмейт iOS'), 2), --Длительность на iOS,
            round(sumIf(hours, usage_platform_ru = 'Букмейт Android'), 2) --Длительность на Android.
    FROM source_db.audition
    WHERE usage_geo_id_name NOT ILIKE '%федеральный округ%' AND usage_country_name = 'Россия'
    GROUP BY name_city
    ORDER BY total_hours DESC
    LIMIT 20;

В результате получили таблицу топ-20 городов и регионов России по суммарному количеству прочитанных и прослушанных часов любого контента с мобильных устройств. Первые три места разделили Москва (52839.85 -общее количество часов прослушивания), Санкт-Петербург(25088.66 -общее количество часов прослушивания) и Московская область(15914.99 -общее количество часов прослушивания). Больше всего контент прослушивают на платформе Android ( Например в Москве - 18573.29 часов), когда как на iOS более чем в два раза больше ( В Москве - 18573.29часов).

    name_city                              |total_hours|
    ---------------------------------------+-----------+
    Москва                                 |   52839.85|
    Санкт-Петербург                        |   25088.66|
    Москва и Московская область            |   15914.99|
    Екатеринбург                           |    9712.36|
    Россия                                 |    8836.88|
    Краснодар                              |    7835.64|
    Новосибирск                            |    7362.47|
    Ростов-на-Дону                         |    6326.94|
    Казань                                 |    5940.37|
    Санкт-Петербург и Ленинградская область|     5805.4|
    Пермь                                  |    5557.12|
    Нижний Новгород                        |    5215.25|
    Уфа                                    |     5213.9|
    Челябинск                              |    5192.26|
    Краснодарский край                     |     4167.1|
    Красноярск                             |    4096.47|
    Воронеж                                |    3681.58|
    Тюмень                                 |    3665.17|
    Самара                                 |    3536.03|
    Чебоксары                              |    3276.23|
    
    

## Задание 2

    SELECT  main_content_name, --Название книги
            main_author_name, --Автор книги
            round(sum(hours), 2) AS total_hours, --Общая длительность прочитанного и прослушанного контента
            round(avgIf(hours, main_content_type = 'Book'), 2) AS avg_read_book, --Cреднее время чтения текстовой книги
            round(avgIf(hours, main_content_type = 'Audiobook'), 2) AS avg_read_audio --Cреднее время прослушивания аудиокниги
    FROM source_db.content as c
    JOIN source_db.audition as a ON c.main_content_id = a.main_content_id
    WHERE usage_platform_ru = 'Букмейт iOS' OR usage_platform_ru = 'Букмейт Android'
    GROUP BY main_content_name, main_author_name
    HAVING COUNT(DISTINCT main_content_type) = 2
    ORDER BY total_hours DESC
    LIMIT 5;

    main_content_name                                     |main_author_name| total_hours| avg_read_book| avg_read_audio|
    ------------------------------------------------------+----------------+-----------+-------------+--------------+
    Илон Маск                                             |Уолтер Айзексон |    1012.93|         0.29|          0.69|
    Железное пламя                                        |Ребекка Яррос   |     781.16|         1.74|          1.89|
    Убийства и кексики.
    Детективное агентство «Благотворительный магазин»     |Питер Боланд    |     541.87|         0.68|          1.63|
    Четвертое крыло                                       |Ребекка Яррос   |     501.67|         1.58|          1.34|
    Земля лишних. Трилогия                                |Андрей Круз     |     481.97|         2.47|          2.75|

    В топ-5 книг по суммарному количеству прочитанных и прослушанных часов на мобильных платформах вошли "Илон Макс" Уолтер Айзексон, "Железное пламя"Ребекка Яррос, "Земля лишних. Трилогия" Андрей Круз, "Убийства и кексики.Детективное агентство «Благотворительный магазин» "Питер Боланд, "Четвертое крыло" Ребекка Яррос. Также вычислите среднее время чтения и прослушивания в зависимости от типа книги: текст или аудио. Выявлено, что охотнее всего пользователи прослушивают эти произведения, нежели читают.

## Задание 3

    SELECT
        main_author_name, --Автор книги
        round(sum(hours), 2) AS total_hours, --Общая длительность прочитанного и прослушанного контента
        countDistinctIf(main_content_id, main_content_type = 'Book') as count_book, --Количество уникальных текстовых книг
            round(avgIf(hours, main_content_type = 'Audiobook' AND usage_platform_ru IN ('Букмейт iOS','Букмейт Android')), 2) AS avg_audio --среднюю длительность прослушивания их аудиокниг только на мобильных устройствах
    FROM source_db.content as c
    JOIN source_db.audition as a ON c.main_content_id = a.main_content_id
    WHERE main_author_name IN (
        -- Авторы, у которых есть аудиокниги
    SELECT DISTINCT main_author_name
    FROM source_db.content
    WHERE main_content_type = 'Audiobook'
    )
    GROUP BY main_author_name
    ORDER BY total_hours DESC
    LIMIT 10;


    main_author_name  |total_hours|count_book|avg_audio|
    ------------------+-----------+----------+---------+
    Андрей Усачев     |   23100.12|12        |     0.51|
    Лиана Шнайдер     |   14373.02|4         |     0.35|
    Анна Казалис      |    5736.92|18        |     0.09|
    Сергей Лукьяненко |    4655.68|54        |     1.76|
    Борис Акунин      |    4555.81|52        |     1.48|
    Коллектив авторов |     3794.0|19        |     0.74|
    Николай Носов     |    3756.55|13        |     0.84|
    Дядя Кузя         |    3739.86|0         |      NaN|
    Анджей Сапковский |    3429.16|14        |     1.25|
    Валентин Постников|    2953.88|1         |     0.49|    
    
    
   Топ-10 авторов по суммарной длительности чтения их книг на всех платформах, включая веб состоит из авторов Андрей Усачев, Лиана Шнайдер, Анна Казалис, Сергей Лукьяненко, Борис Акунин, Коллектив авторов, Николай Носов, Дядя Кузя, Анджей Сапковский, Валентин Постников. Автор с максимальным количеством уникальных текстовых книг (тип контента 'Book' ) Сергей Лукьяненко (54 книг) и Борис Акунин (52 книг). По средней длительности прослушивания аудиокниг только на мобильных устройствах лидирует Андрей Усачев(23100.12 часов) и Лиана Шнайдер(14373.02 часов).

## Задание 4

    WITH 
    user_platform_stats AS (
    SELECT
        puid, -- Пользователь
        usage_platform_ru as platform, -- Определяем платформу
        SUM(hours) as total_hours,  -- Общая сумма часов
        SUMIf(hours, main_content_type = 'Audiobook') as audio_hours, -- Время прослушивания книг 
        SUMIf(hours, main_content_type = 'Book') as book_hours -- Время чтения книг
    FROM source_db.audition a
    JOIN source_db.content c ON a.main_content_id = c.main_content_id
    WHERE hours > 0 
      AND (usage_platform_ru = 'Букмейт iOS' OR usage_platform_ru = 'Букмейт Android') -- Добавлены скобки
    GROUP BY puid, platform 
    HAVING total_hours > 0  -- Исключаем пользователей без сессий
    ),
    user_segments AS (
    SELECT
        puid, -- Пользователь
        argMax(platform, total_hours) as main_platform, -- Платформа
        multiIf(
            audio_hours / total_hours >= 0.7, 'Слушатель', 
            book_hours / total_hours >= 0.7, 'Читатель', 
            'Оба'
        ) AS user_segment -- Группа
    FROM user_platform_stats
    GROUP BY puid, user_segment
    )
    SELECT
    main_platform, -- Платформа
    user_segment, -- Группа пользователя
    COUNT(*) as user_count -- Количество пользователей в группе
    FROM user_segments
    GROUP BY main_platform, user_segment
    ORDER BY main_platform, user_count DESC;

    main_platform  |user_segment|user_count|
    ---------------+------------+----------+
    Букмейт Android|Читатель    |2254      |
    Букмейт Android|Слушатель   |2084      |
    Букмейт Android|Оба         |707       |
    Букмейт iOS    |Читатель    |1933      |
    Букмейт iOS    |Слушатель   |1479      |
    Букмейт iOS    |Оба         |432       |
    
 Предположение менеджера о том,что среди пользователей Android примерно одинаковое количество читателей и слушателей, а на устройствах iOS читателей книг вдвое больше, чем слушателей оказалось почти верным. Среди пользователей Android примерно одинаковое количество читателей(2254) и слушателей(2084). А на устройствах iOS читателей книг(1933) больше, чем слушателей на треть(1479). 

## Задание 5

    WITH hours_day_week AS (SELECT
        usage_platform_ru,
        main_content_type, --Тип контента (бумажная книга иили аудиозапись)
        sum(hours) as hours_day, --Среднее время прослушивания
        multiIf(toDayOfWeek(msk_business_dt_str) >= 1 AND toDayOfWeek(msk_business_dt_str) <= 5,'Рабочий день', toDayOfWeek(msk_business_dt_str) >= 6 AND toDayOfWeek(msk_business_dt_str) <= 7, 'Выходной', 'Нет') AS day_week --День
    FROM source_db.audition as a
    JOIN source_db.content c ON a.main_content_id = c.main_content_id
    WHERE (usage_platform_ru = 'Букмейт iOS' OR usage_platform_ru = 'Букмейт Android' OR usage_platform_ru = 'Букмейт Web') AND main_content_type != 'Comicbook'
    GROUP BY day_week, usage_platform_ru, main_content_type)
    SELECT 
        usage_platform_ru,
        main_content_type, --Тип контента (бумажная книга иили аудиозапись)
        round((avg(hours_day)), 0) AS avg_hours,
        day_week
    FROM hours_day_week
    GROUP BY day_week, main_content_type, usage_platform_ru
    ORDER BY  usage_platform_ru DESC, main_content_type DESC;

    usage_platform_ru|main_content_type|avg_hours|day_week    |
    -----------------+-----------------+---------+------------+
    Букмейт iOS      |Book             |  17327.0|Рабочий день|
    Букмейт iOS      |Book             |   6283.0|Выходной    |
    Букмейт iOS      |Audiobook        |   9540.0|Выходной    |
    Букмейт iOS      |Audiobook        |  29667.0|Рабочий день|
    Букмейт Web      |Book             |   1160.0|Выходной    |
    Букмейт Web      |Book             |   3413.0|Рабочий день|
    Букмейт Web      |Audiobook        |   1020.0|Выходной    |
    Букмейт Web      |Audiobook        |   2791.0|Рабочий день|
    Букмейт Android  |Book             |  36482.0|Рабочий день|
    Букмейт Android  |Book             |  14233.0|Выходной    |
    Букмейт Android  |Audiobook        |  70338.0|Рабочий день|
    Букмейт Android  |Audiobook        |  23362.0|Выходной    |

Связь между форматом использования приложения (прослушивание или чтение) и днём недели явно видна. Среднее количество часов прослушивания контента в день выше в рабочие дни. Как видно из результатов расчётов, использование аудиокниг в выходные на всех платформах, включая веб падает примерно в три раза. Возможно, это говорит о том, что выходной день люди занимаются другими делами, кторым не могут уделить внимание на рабочей неделе. 

## Задание 6

    WITH platform_max_versions AS (
    SELECT 
        usage_platform_ru,
        max(app_version) as max_version
    FROM source_db.audition as a
    JOIN source_db.content c ON a.main_content_id = c.main_content_id
    WHERE usage_platform_ru IN ('Букмейт iOS', 'Букмейт Android')
      --AND app_version IS NOT NULL
    GROUP BY usage_platform_ru
    )
    SELECT
        a.usage_platform_ru, --Платформа
        countDistinct(puid) AS count_users, --Уникальное количество пользователей
        countDistinctIf(a.puid, app_version = pmv.max_version) AS count_max_version, --Количество пользователей с максимальным обновлением приложения
        round((countDistinctIf(a.puid, app_version = pmv.max_version)/countDistinct(puid)), 2) as part_count --Доля пользователей на платформе с обнавлением
    FROM source_db.audition as a
    JOIN source_db.content c ON a.main_content_id = c.main_content_id
    JOIN platform_max_versions pmv ON a.usage_platform_ru = pmv.usage_platform_ru
    GROUP BY a.usage_platform_ru;

    a.usage_platform_ru|count_users|count_max_version|part_count|
    -------------------+-----------+-----------------+----------+
    Букмейт iOS        |3915       |75               |      0.02|
    Букмейт Android    |5109       |1490             |      0.29|

Предположение, о том, что больший процент пользователей iOS используют последнюю версию приложения и в целом чаще его обновляют, является неверной. Из расчётов видно, что количество пользователей с максимальным обновлением на Android больше(1490 пользователей), чем на iOS (75 пользователей). 

## Задание 7

    WITH updates AS (
    SELECT puid,
           usage_platform_ru,
           app_version,
           msk_business_dt_str,
           lagInFrame(app_version) OVER (PARTITION BY puid ORDER BY msk_business_dt_str) AS previous_version
    FROM source_db.audition
    )
    SELECT usage_platform_ru, --Платформа
       countDistinct(puid) AS active_users, --Количество уникальных пользователей с обновлением
       countIf(not isNull(previous_version)) AS total_updates, --Количество обновлений
       round(COALESCE(countIf(not isNull(previous_version)) / countDistinct(puid), 0), 2) AS update_rate --Средняя частота обновлений на пользователя
    FROM updates
    WHERE usage_platform_ru = 'Букмейт iOS' OR usage_platform_ru ='Букмейт Android' OR usage_platform_ru ='Букмейт Web'
    GROUP BY usage_platform_ru
    ORDER BY update_rate DESC;

    usage_platform_ru|active_users|total_updates|update_rate|
    -----------------+------------+-------------+-----------+
    Букмейт Android  |5124        |118458       |      23.12|
    Букмейт iOS      |3935        |70285        |      17.86|
    Букмейт Web      |1470        |1742         |       1.19|

Как видно из расчётов пользователи обновляют приложение на Букмейт Android (118458), Букмейт iOS(70285) и Букмейт Web(1742). Чаще всего обнавляют версии на Android, средняя частота обновлений на пользователя около 23. Когда как на iOS - около 18 раз. На Букмейт Web составило одно обновление на пользователя.

## Задание 8

    SELECT 
         countDistinct(main_content_id) --Считаем количество уникальных книг с тегом
    FROM source_db.content 
    WHERE has(published_topic_title_list, 'Магия'); --Ищем в массиве книги с тегом 'Магия'

    count_book_magic|
    ----------------+
    46              |

## Задание 9

    SELECT 
        countDistinct(main_content_id) as book_count --Количество книг
    FROM source_db.content 
    WHERE (main_content_name ILIKE '%магия%') AND
        --Без тега "Магия"
    NOT has(published_topic_title_list, 'Магия') AND
        --Без тега "Художественная литература"
     NOT has(published_topic_title_list, 'Художественная литература');

    book_count|
    ----------+
    49        |

## Задание 10

    SELECT 
        round(AVG(length(published_topic_title_list)), 2) AS avg_all_records,  --Cреднее количество категорий у книг в каталоге в целом
        round(AVG(length(published_topic_title_list)) FILTER(WHERE has(published_topic_title_list, 'Магия')), 2) AS avg_magic_records  --Среднее количество категорий у книг с тегом «Магия»
    FROM source_db.content
    WHERE main_content_type = 'Book';

    avg_all_records|avg_magic_records|
    ---------------+-----------------+
               3.27|             2.93|

    Cреднее количество категорий у книг с тегом «Магия» 2.93 и среднее количество категорий у книг в каталоге в целом 3.27. По расчётам у книг с тегом «Магия» количество тегов не привышает 3-4, как и среднее количество категорий в каталоге.

## Задание 11

    SELECT 
        usage_country_name, --Название страны
        usage_platform_ru, --Название платформы
        ROUND(STDDEV_POP(hours_sessions_long) / AVG(hours_sessions_long), 2) AS variation_coefficient --Коэффициент вариации
    FROM source_db.audition AS a
    JOIN source_db.content c ON a.main_content_id = c.main_content_id
    WHERE hours_sessions_long > 0
    GROUP BY usage_country_name, usage_platform_ru
    ORDER BY variation_coefficient DESC
    LIMIT 1;

    usage_country_name|usage_platform_ru|variation_coefficient|
    ------------------+-----------------+---------------------+
    Латвия            |Букмейт Android  |                 6.98|
  
Как видно из расчётов, длина пользовательской сессии (поле hours_sessions_long ) записывается некорректно минимум в одной стране. В Латвии и на Букмейт Android видна аномалия данных - коэффициент вариации здесь составляет 6.98. Эта наибольшее значение из всех стран.

# Вывод

    Первые три места по количеству прослушиваний разделили Москва (52839.85 -общее количество часов прослушивания), Санкт-Петербург(25088.66 -общее количество часов прослушивания) и Московская область(15914.99 -общее количество часов прослушивания). Больше всего контент прослушивают на платформе Android ( Например в Москве - 18573.29 часов), когда как на iOS более чем в два раза больше ( В Москве - 18573.29часов).
    В топ-5 книг по суммарному количеству прочитанных и прослушанных часов на мобильных платформах вошли "Илон Макс" Уолтер Айзексон, "Железное пламя"Ребекка Яррос, "Земля лишних. Трилогия" Андрей Круз, "Убийства и кексики.Детективное агентство «Благотворительный магазин» "Питер Боланд, "Четвертое крыло" Ребекка Яррос.
    Топ-10 авторов по суммарной длительности чтения их книг на всех платформах, включая веб состоит из авторов Андрей Усачев, Лиана Шнайдер, Анна Казалис, Сергей Лукьяненко, Борис Акунин, Коллектив авторов, Николай Носов, Дядя Кузя, Анджей Сапковский, Валентин Постников. Автор с максимальным количеством уникальных текстовых книг (тип контента 'Book' ) Сергей Лукьяненко (54 книг) и Борис Акунин (52 книг). По средней длительности прослушивания аудиокниг только на мобильных устройствах лидирует Андрей Усачев(23100.12 часов) и Лиана Шнайдер(14373.02 часов).
    Предположение менеджера о том,что среди пользователей Android примерно одинаковое количество читателей и слушателей, а на устройствах iOS читателей книг вдвое больше, чем слушателей оказалось почти верным. Среди пользователей Android примерно одинаковое количество читателей(2254) и слушателей(2084). А на устройствах iOS читателей книг(1933) больше, чем слушателей на треть(1479).
    Среднее количество часов прослушивания контента в день выше в рабочие дни.
    Количество пользователей с максимальным обновлением на Android больше(1490 пользователей), чем на iOS (75 пользователей). Пользователи обновляют приложение на Букмейт Android (117073), Букмейт iOS(69443) и Букмейт Web(1733). Чаще всего обнавляют версии на Android, средняя частота обновлений на пользователя около 23. Когда как на iOS - около 18 раз. На Букмейт Web составило одно обновление на пользователя.
    Всего книг с тегом "Магия" 46, а книг с названием содержащим слово "Магия" 49. Три книги не имеют тега магия, хотя в названии книги оно есть. 
    По расчётам у книг с тегом «Магия» количество тегов не привышает 3-4, как и среднее количество категорий в каталоге.
     В Латвии и на Букмейт Android видна наибольшая аномалия данных - коэффициент вариации здесь составляет 6.98.